In [ ]:
import os
import shutil
from Libs.Shell import run_shell
from pathlib import Path
import xml.etree.ElementTree as ET
from tqdm import tqdm
from zipfile import ZipFile

In [ ]:
run_shell("apktool empty-framework-dir --force")

In [ ]:
run_shell('adb wait-for-device')
run_shell('adb devices')

In [ ]:
#https://github.com/wtsxDev/android-security-list

In [ ]:
def getPackageApkPaths(package):
    return [line.lstrip("package:") for line in  run_shell(f"adb shell pm path {package}")]

In [ ]:
#Downloads dir
#run_shell('adb shell ls /storage/emulated/0/Download')
#run_shell('adb shell ls /sdcard/Download')

In [ ]:
class Apk():
    def __init__(self,package, file, log=None):
        self.package = package
        self.file = file
        self.log = log
        self.basename = file.split('/')[-1]
        self.decompiled = False
        self.downloaded  = Path(os.path.join(
            'phone_sync_dir',
            self.package,
            Path(file).name
        )).exists()
        self.package_extraction_dir = os.path.join(
            'phone_sync_dir',
            self.package,
            Path(file).stem
        )
        self.update_decompiled_flag()
    def findString(self,string):
        for file in Path(self.package_extraction_dir).glob('**/*'):
            if file.is_file():
                try:
                    if string in file.read_text():
                        yield file
                except Exception as e:
                    pass
    def update_decompiled_flag(self):
         self.decompiled = Path(self.package_extraction_dir).exists()

    def decompile(self):
        self.update_decompiled_flag()
        if self.decompiled:
            raise Exception("Already Decompiled")
        return run_shell(f'apktool d -o {self.package_extraction_dir} {self.file}')
    
    def make_ready_compile(self):
        self.update_decompiled_flag()
        if not self.decompiled:
            raise Exception("Not Decompiled Yet")
        with open(os.path.join(self.package_extraction_dir,'AndroidManifest.xml'),'r')as f:
            xml = ET.fromstring(f.read())
            print(dir(xml))
            
    def download(self, local_path):
        if not downloaded:
            log = run_shell(f"adb pull {self.file} {local_path}")
            self.file  = local_path
            self.downloaded  = True

    def recompile(self):
        self.update_decompiled_flag()
        if not self.decompiled:
            raise Exception("Not Decompiled Yet")
        new_file = Path(self.file)
        new_file = new_file.with_name(new_file.stem+'_recompiled'+new_file.suffix)
        compile_log = run_shell(f'apktool b -o {new_file} {self.package_extraction_dir}')
        return Apk(package=self.package, file=str(new_file), compile_log=compile_log)

    def __repr__(self):
        self.update_decompiled_flag()
        dcs = ''
        if self.decompiled:
            dcs = " Decompiled"
        return f"[LocalApk{dcs} {self.basename}]"

In [ ]:
class ApkFindResult():
    def __init__(self, apk, file):
        self.apk = apk
        self.file = file
    def __repr__(self):
        return f'{self.apk} - {self.file}'

In [ ]:
class ApkCollection():
    def __init__(self, apk_list):
        self.apk_list = apk_list
    def extend(self,other):
        self.apk_list.extend(other)
    def __iter__(self):
        return iter(self.apk_list)
    def findString(self, string):
        for apk_file in self.apk_list:
            for file in apk_file.findString(string):
                yield ApkFindResult(apk=apk_file, file=file)

In [ ]:
class App():
    def __init__(self, package):
        self.package = package
        self.package_cache_dir = os.path.join(
            'phone_sync_dir',
            self.package
        )
        self.package_main_activity = run_shell(f'adb shell cmd package resolve-activity {self.package}',extraction='.*name=(.*)')[0][0]
        self.Apks = ApkCollection([])
        self.getApks()
        
    def force_stop(self):
        return run_shell(f"adb shell am force-stop {self.package}")
    
    def start_main_appliacation(self):
        return run_shell(f'adb shell am start {self.package}/{self.package_main_activity}')
    
    def getApks(self):
        self.Apks.extend([Apk(file=apk, package=self.package) for apk in getPackageApkPaths(self.package)])
        
    def decompileAllApks(self):
        for file in self.Apks:
            try:
                file.decompile()
            except Exception as e:
                print(e)
                
    def downloadAllApks(self):
        if not os.path.exists(self.package_cache_dir):
            os.mkdir(self.package_cache_dir)
          
        for file in self.Apks:
            local_path = os.path.join(self.package_cache_dir, file.basename)
            file.download(local_path)
        return ApkCollection(self.Apks)
    
    def uninstall_leaving_data(self):
        return run_shell(f"adb shell pm uninstall -k {self.package}")
    
    def install(self):
        if len(self.localApks)>1:
            return self.install_apk_bundle()
        else:
            return run_shell(f"adb install {self.localApks[0].file}")
    def install_apk_bundle(self):
        self.uninstall_leaving_data()
        tmp_files = []
        tot_bytes = 0
        
        for file in self.localApks:
            remote_filename = file.basename
            remote_filepath = f'/data/local/tmp/{remote_filename}'
            tmp_files.append(remote_filepath)
            run_shell(f'adb push {file.file} {remote_filepath}')
            tot_bytes += int(run_shell(f'adb shell du -b {remote_filepath}',split_other='\t',splitlines=False)[0])

        session_id = run_shell(f'adb shell pm install-create -S {tot_bytes}',extraction='.*\[(.*)\]')[0][0]
        for index, file in enumerate(tmp_files):
            file_size = int(run_shell(f'adb shell du -b {file}',split_other='\t',splitlines=False)[0])
            run_shell(f'adb shell pm install-write -S {file_size} {session_id} {index} {file}')
        run_shell(f'adb shell pm install-commit {session_id}')

        for file in tmp_files:
            run_shell(f'adb shell rm {file}')

        
    def __repr__(self):
        return f"[App {self.package}]\n\tRemote:{', '.join(map(repr,self.remoteApks))}\n\tLocal:{', '.join(map(repr,self.localApks))}"

In [ ]:
def getAllAppPackageNames():
    return run_shell('adb shell pm list packages -3 |cut -f 2 -d ":"')

In [89]:
def findPackageNameByPartial(part):
    for line in getAllAppPackageNames():
        if part in line:
            return(line)

In [ ]:
#run_shell("adb shell ps -A")

In [ ]:
app = App('droidhang.twgame.restaurant')

In [ ]:
app.start_main_appliacation()

In [ ]:
app.decompileAllApks()

In [ ]:
for result in app.Apks.findString("test"):
    print(result)

In [ ]:
run_shell(f'adb shell dumpsys package | grep -Eo "^[[:space:]]+[0-9a-f]+[[:space:]]{app.package}/[^[:space:]]+" | grep -oE "[^[:space:]]+$"')

In [ ]:
compiled_apk=[apk.recompile() for apk in apks]


In [ ]:
type(compiled_apk)

In [ ]:
app.install()

In [ ]:
compiled_apk.sign("Mykey.pem")

In [ ]:
compiled_apk.install()

In [ ]:
apk.make_ready_compile()

In [ ]:
app.package

In [ ]:
run_shell(f"adb shell am start -n {app.package}/{app.main_activity}")

In [ ]:

with open(os.path.join('phone_sync_dir/net.kairosoft.android.pool/base','AndroidManifest.xml'),'r')as f:
    tree = ET.fromstring(f.read())
    root = tree.get


item = xml.find('application')
for i in item.attrib:
    if 'debuggable' in i:
        item.attrib[i]=True

with open(os.path.join('phone_sync_dir/net.kairosoft.android.pool/base','AndroidManifest.xml'),'w')as f:
    tree.